# Contact De Duplication

In [2]:
import src.util.helper as helper
import src.settings.constants as const
import src.webscrapper.webscrapper as wsc
import pandas as pd
import re

In [4]:
url_list = ["Bitnomial",
"BlackEdge Capital",
"Blue Trading",
"Candor",
"CapFund Group"]

list_rsp = []
for url in url_list:
	rsp = {}
	website = wsc.get_company_website(url)
	rsp["website"] = website
	rsp["company"] = url
	list_rsp.append(rsp)

df = pd.DataFrame.from_dict(list_rsp)
df.head()

2024-Sep-26 16:31:30 - WebScrapper - INFO - For company bitnomial found website https://bitnomial.com.
2024-Sep-26 16:31:31 - WebScrapper - INFO - For company blackedge capital found website https://www.blackedge.com.
2024-Sep-26 16:31:33 - WebScrapper - INFO - For company blue trading found website https://www.bluetradingsystems.com.
2024-Sep-26 16:31:34 - WebScrapper - INFO - For company candor found website https://www.candorcs.org.
2024-Sep-26 16:31:35 - WebScrapper - INFO - For company capfund group found website https://www.capfundinc.com.


,website,company
0,https://bitnomial.com,Bitnomial
1,https://www.blackedge.com,BlackEdge Capital
2,https://www.bluetradingsystems.com,Blue Trading
3,https://www.candorcs.org,Candor
4,https://www.capfundinc.com,CapFund Group


In [ ]:
rsp1 = wsc.get_company_website_and_legal("A.S.R.")

In [ ]:
def extract_legal_name(text:str):
	pattern = r'\b[A-Za-z0-9\s,]+?\s(Inc\.|LLC|Ltd\.|Corp\.|Corporation|Co\.|Limited|LLP|L\.P\.|PLC|GmbH|S\.A\.|Pty\. Ltd\.|BV)\b'
	match = re.search(pattern, text)
	if match:
			return match.group(0)
	else:
			return None
text = "ASR Group International, Inc. is the world’s largest refiner and marketer of cane sugar with an annual production capacity of 6 million metric tons of sugar. We produce a full line of grocery, industrial, foodservice and specialty sweetener products, with a strong focus on innovation and product development. We hold leading positions in our core markets, which include some of the largest end-markets for sweeteners in the world."
print(extract_legal_name(text))

In [7]:
from urllib.robotparser import RobotFileParser

def is_scraping_allowed_by_robots_file(url, robots_text, user_agent='*'):
	# Initialize and parse the robots.txt
	rp = RobotFileParser()
	
	print(rp.parse(robots_text.splitlines()))
	# Check if the website allows scraping for the given user agent
	return rp.can_fetch(user_agent, url)

is_scraping_allowed_by_robots_file(url='https://seasideequity.com/team/andrew-thompson/',robots_text='seasideequity')

None


True

In [2]:
import src.util.log_helper as log_helper
import src.settings.constants as const
import src.util.helper as helper
import src.util.sqllite_helper as db_manager
import src.util.api_helper as api_helper
import src.util.proxycurl as pcurl
from bs4 import BeautifulSoup
from bs4.element import Comment
from rapidfuzz import fuzz
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser
import pandas as pd
import re
import time
import io
import json
import random
import requests
from src.util.similarity_helper import get_fuzzy_similarity

def get_website_robot_file(website_url : str, company_name :str):
	
	robot_file = ""
	str_sql = "Select robot_file  from company_master Where website = ? COLLATE NOCASE"
	robot_file = db_manager.select_scaler(str_sql,[website_url])
	if robot_file != const.NOT_EXISTS:
		#logger.info(f'company {company_name} and robot file exists in db.')
		return robot_file

	# As Robot file does not exists in DB, Download from Website and Insert into DB
	try:
		with requests.get(website_url + "/robots.txt") as r:
			r.raise_for_status()
			file_content =  io.StringIO(r.text)
			for line in file_content:
				if robot_file == const.NOT_EXISTS:
					robot_file = line
				else:
					robot_file += "\n" + line	
	except Exception as e:
		#logger.exception(e)
		robot_file = const.NOT_FOUND

	# Now Insert Robot file and other content into DB
	str_sql = "Insert into company_master ( company_name, website,robot_file) Values(?,?,?)"
	db_manager.execute_sql(str_sql,[company_name,website_url,robot_file])

	return robot_file

In [3]:
robat_files = get_website_robot_file(website_url='https://www.maximgrp.com/about	',company_name='seasideequity')

In [4]:
import src.util.log_helper as log_helper
import src.settings.constants as const
import src.util.helper as helper
import src.util.sqllite_helper as db_manager
import src.util.api_helper as api_helper
import src.util.proxycurl as pcurl
from bs4 import BeautifulSoup
from bs4.element import Comment
from rapidfuzz import fuzz
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser
import pandas as pd
import re
import time
import io
import json
import random
import requests
from src.util.similarity_helper import get_fuzzy_similarity
import src.webscrapper.webscrapper as web
def get_page_content(company_name:str,page_url:str, robot_file ):
	"""
	This Function will go through Website specific page Find page text content.
	"""
	rsp = {}
	rsp["is_success"] = False

	str_sql = "SELECT page_text from company_pages where page_url= ? and company_name=?"
	res = db_manager.select_scaler(str_sql,[page_url,company_name])
	if res != const.NOT_EXISTS:
		rsp["page_text"]= res
		rsp["is_success"] = True
		return rsp

	if robot_file == const.NOT_FOUND:
		is_url_allowed_to_scrap = True
	else:
		is_url_allowed_to_scrap = is_scraping_allowed_by_robots_file(page_url,robot_file)

	if not is_url_allowed_to_scrap:
		#logger.info(f"url {page_url} is not allowed scrap by robot.txt file.")
		rsp["page_text"] = "SCRAPPING_NOT_ALLOWED"	
		return rsp
	
	#logger.info(f"Request started for : {page_url}")
	try:
		headers = {'User-Agent': random.choice(const.USER_AGENTS)}
		with requests.get(page_url,headers=headers, timeout=const.REQUEST_TIME_OUT) as r:
			r.raise_for_status()
			rsp["page_text"] = helper.clean_scrapped_text(web.text_from_html(r.text)) 
			rsp["is_success"] = True
	except Exception as e:
		rsp["page_text"] = "ERROR" 
		rsp["is_success"] = False
		#logger.error(f"error occured : {e}")
	#logger.info(f"Request Completed for : {page_url}")
	if rsp["is_success"]:
		url_type = web.get_url_type(page_url)
		str_sql="INSERT INTO company_pages (company_name, page_url, page_text,url_type) VALUES (?, ?, ?,?)"
		db_manager.execute_sql(str_sql,[company_name,page_url,rsp["page_text"],url_type])		
	return rsp

In [5]:
rsp = get_page_content("maxim group","https://www.maximgrp.com/about",robat_files)

In [6]:
rsp['page_text']

''